In [1]:
import pulp

In [2]:
import numpy as np
import pandas as pd
import math

In [3]:
members = range(1,4)

In [4]:
days = range(1,9)

In [5]:
# defining the variables
x = pulp.LpVariable.dicts("x",(members,days),0,1,pulp.LpInteger)

In [6]:
print(x)

{1: {1: x_1_1, 2: x_1_2, 3: x_1_3, 4: x_1_4, 5: x_1_5, 6: x_1_6, 7: x_1_7, 8: x_1_8}, 2: {1: x_2_1, 2: x_2_2, 3: x_2_3, 4: x_2_4, 5: x_2_5, 6: x_2_6, 7: x_2_7, 8: x_2_8}, 3: {1: x_3_1, 2: x_3_2, 3: x_3_3, 4: x_3_4, 5: x_3_5, 6: x_3_6, 7: x_3_7, 8: x_3_8}}


In [7]:
# objective
model = pulp.LpProblem("Warmup", pulp.LpMinimize)

# objective function : minimize the number of warmups each member has to do
for m in members:
    model += pulp.lpSum(x[m][d] for d in days)

C:\Users\luket\AppData\Local\Programs\Python\Python310\lib\site-packages\pulp\pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [8]:
# every member has run to the warmup at least once
for m in members:
    model += pulp.lpSum(x[m][d] for d in days) >= 1

In [9]:
# nobody should have to run the warmup more than total/members rounded up
for m in members:
    model += pulp.lpSum(x[m][d] for d in days) <= math.ceil(len(days)/len(members))

In [10]:
# every warmup must have a leader
for d in days:
    model += pulp.lpSum(x[m][d] for m in members) == 1

In [11]:
# number of sessions between running warmups : 1
for m in members:
    for d in days:
        if d>1:
            model += x[m][d] + x[m][d-1] <= 1

In [12]:
# number of sessions between running warmups : 2
for m in members:
    for d in days:
        if d>2:
            model += x[m][d] + x[m][d-1] + x[m][d-2] <= 1

## Solving the Model

In [13]:
# Solve ILP problem and post-processing to get the summary
model.solve()

1

In [14]:
# model

In [15]:
for vi in model.variables():
    if vi.varValue == 1:
        print(f"{vi.name} = {vi.varValue:.0f}")

x_1_2 = 1
x_1_5 = 1
x_1_8 = 1
x_2_1 = 1
x_2_4 = 1
x_2_7 = 1
x_3_3 = 1
x_3_6 = 1


## Converting to a dataframe for a clean output

In [27]:
myperson = []
myday = []
value = []

for vi in model.variables():
    person = vi.name.split("_")[1]
    day = vi.name.split("_")[2]
    
    myperson.append(person)
    myday.append(day)
    value.append(vi.varValue)
    
data = {'Day':myday, 'Person':myperson, 'Value':value} 

df = pd.DataFrame(data)

In [34]:
df.sort_values(by='Day', ascending=True)

,Day,Person,Value,Name
0,1,1,0.0,Dan
16,1,3,0.0,Jacinta
8,1,2,1.0,Steven
1,2,1,1.0,Dan
17,2,3,0.0,Jacinta
9,2,2,0.0,Steven
2,3,1,0.0,Dan
18,3,3,1.0,Jacinta
10,3,2,0.0,Steven
19,4,3,0.0,Jacinta


In [28]:
# df.to_csv('role_optimazation.csv', index = False)

In [29]:
df

,Day,Person,Value
0,1,1,0.0
1,2,1,1.0
2,3,1,0.0
3,4,1,0.0
4,5,1,1.0
5,6,1,0.0
6,7,1,0.0
7,8,1,1.0
8,1,2,1.0
9,2,2,0.0


In [30]:
person = [
    'Dan',
'Steven',
'Jacinta',
]

In [31]:
person_dict = {}
for i in enumerate(person):
    person_dict[i[0] + 1] = i[1]

In [32]:
df['Name'] = df['Person'].apply(lambda x:person_dict[int(x)])

In [33]:
df

,Day,Person,Value,Name
0,1,1,0.0,Dan
1,2,1,1.0,Dan
2,3,1,0.0,Dan
3,4,1,0.0,Dan
4,5,1,1.0,Dan
5,6,1,0.0,Dan
6,7,1,0.0,Dan
7,8,1,1.0,Dan
8,1,2,1.0,Steven
9,2,2,0.0,Steven


In [23]:
def mapper(row):
    return str(int(row['Value']))

In [24]:
df['y'] = df.apply(mapper,axis = 1)

In [25]:
df

,Person,Day,Value,Name,y
0,1,1,0.0,Dan,0
1,1,2,1.0,Dan,1
2,1,3,0.0,Dan,0
3,1,4,0.0,Dan,0
4,1,5,1.0,Dan,1
5,1,6,0.0,Dan,0
6,1,7,0.0,Dan,0
7,1,8,1.0,Dan,1
8,2,1,1.0,Steven,1
9,2,2,0.0,Steven,0


In [26]:
# df.to_csv('role_optimization_final_2.csv', index = False)

In [37]:
len(days)

8